In [1]:
from pprint import pprint
import pandas as pd

In [2]:
from requests_futures.sessions import FuturesSession

In [ ]:
from tracking_grants import articles_f, pmid_f, email, ncbi_api_key, tool_name

In [4]:
from ratelimit import limits, sleep_and_retry
from tqdm.auto import tqdm
import re

In [12]:
articles = pd.read_csv(articles_f)

In [20]:
articles.nunique()

reference_id        8164
grant_id            2232
program                4
score               7927
DOI                 6923
ISSN                1175
journal_name        1273
publisher            147
coci_citations       440
references           242
cr_subject           488
authors_count         43
created               19
deposited             14
indexed                2
published-online      22
issued                24
pmid                5756
dtype: int64

In [21]:
5756/6923

0.8314314603495594

In [15]:
pmids = pd.read_csv(pmid_f)

In [26]:
pmids['rate_limited'] = pmids.response.str.contains("API rate limit exceeded")

In [31]:
pmids[pmids.response.isna()]

,DOI,pmid,response,ts,rate_limited
58,10.1007/s11010-005-9094-2,NaN,NaN,2020-06-25T22:04:35.626327,NaN
399,10.4252/wjsc.v2.i5.114,NaN,NaN,2020-06-25T22:05:07.967028,NaN
441,10.1006/excr.2001.5234,NaN,NaN,2020-06-25T22:05:12.181530,NaN
890,10.1016/j.jpainsymman.2009.09.013,NaN,NaN,2020-06-25T22:05:56.092379,NaN
983,10.1006/viro.2002.1510,NaN,NaN,2020-06-25T22:06:05.306784,NaN
1008,10.1158/0008-5472.can-03-3500,NaN,NaN,2020-06-25T22:06:08.333998,NaN
1084,10.1186/gb-2008-9-5-r83,NaN,NaN,2020-06-25T22:06:15.496650,NaN
1208,10.1371/journal.pone.0013751,NaN,NaN,2020-06-25T22:06:28.636420,NaN
1255,10.1016/j.ymgme.2004.07.014,NaN,NaN,2020-06-25T22:06:32.650274,NaN
1533,10.1038/onc.2008.250,NaN,NaN,2020-06-25T22:07:00.259965,NaN


In [28]:
pmids[pmids.rate_limited]

ValueError: cannot mask with array containing NA / NaN values

In [25]:
pmids[pmids.pmid.isna()]

,DOI,pmid,response,ts
6,10.1200/jco.2018.36.6_suppl.234,NaN,"<?xml version=""1.0"" encoding=""UTF-8"" ?>\n<!DOC...",2020-06-25T21:48:42.702866
29,10.1200/jco.2007.25.18_suppl.5134,NaN,"<?xml version=""1.0"" encoding=""UTF-8"" ?>\n<!DOC...",2020-06-25T22:04:33.613308
31,10.1200/jco.2008.26.15_suppl.5018,NaN,"<?xml version=""1.0"" encoding=""UTF-8"" ?>\n<!DOC...",2020-06-25T22:04:33.613372
40,10.1109/tns.2011.2136387,NaN,"<?xml version=""1.0"" encoding=""UTF-8"" ?>\n<!DOC...",2020-06-25T22:04:33.613468
51,10.1118/1.2241679,NaN,"<?xml version=""1.0"" encoding=""UTF-8"" ?>\n<!DOC...",2020-06-25T22:04:34.613476
...,...,...,...,...
6883,10.1074/jbc.m108255200,NaN,"{""error"":""API rate limit exceeded"",""api-key"":""...",2020-06-25T22:16:06.741685
6884,10.1016/s1063-5823(06)59019-2,NaN,"<?xml version=""1.0"" encoding=""UTF-8"" ?>\n<!DOC...",2020-06-25T22:16:06.741708
6893,10.1118/1.4925550,NaN,"<?xml version=""1.0"" encoding=""UTF-8"" ?>\n<!DOC...",2020-06-25T22:16:07.741711
6915,10.1002/cncr.20905,NaN,"{""error"":""API rate limit exceeded"",""api-key"":""...",2020-06-25T22:16:09.741804


In [6]:
dois = articles.DOI.sample(100).unique().tolist()

### Async calls with request futures

In [79]:
NCBI_CALLS_PER_SEC = 3

class Eutils:
    def __init__(self, tool, email, api_key):
        self.search_api = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
        self.session = FuturesSession()
        self.session.hooks['response'] = self.response_hook
        
        self.params = {
            "tool": tool,
            "email": email,
            "api_key": api_key,
            "db": "pubmed",
            "retmax": 1,
        }

    @sleep_and_retry
    @limits(calls=NCBI_CALLS_PER_SEC, period=1)
    def search(self, doi):
        params = self.params
        params["term"] = doi

        future = self.session.get(self.search_api, params=params)
        return future
    
    @staticmethod
    def response_hook(resp, *args, **kwargs):
        if resp.status_code != 200:
            resp.data = f"Error:{response.status_code}"
        else:
            text = resp.text
            resp.data = None

            # Only return DOI has been found in the text
            if 'PhraseNotFound' not in text:
                count = int(re.search(r"<Count>(\d+)<\/Count>", text).group(1))
                # Only return unique matches
                if count == 1:
                    match = re.search(r"<Id>(\d+)<\/Id>", text)
                    if match:
                        resp.data =  match.group(1)

In [80]:
eutils = Eutils(tool_name, email, ncbi_api_key)

In [81]:
futures = []
for doi in tqdm(dois):
    futures.append((doi, eutils.search(doi)))

In [82]:
results = {}
for doi, future in tqdm(futures):
    response = future.result()
    results[doi] = response.data

### Async Code with aiohttp

In [7]:
import aiohttp
import asyncio
from asyncio_throttle import Throttler

In [8]:
import time

In [9]:
COUNT_REGEX = r"<Count>(\d+)<\/Count>"
ID_REGEX = r"<Id>(\d+)<\/Id>"

class Eutils():
    def __init__(self, tool, email, api_key, loop, calls_per_sec=3):
        self.baseurl = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
        self.params = {
            "tool": tool,
            "email": email,
            "api_key": api_key,
            "db": "pubmed",
            "retmax": 1,
        }
        
        self.loop = loop
        self.throttler = Throttler(rate_limit=calls_per_sec, period=1)
    
    async def __fetch(self, session, params):
        async with self.throttler:
            async with session.get(self.baseurl, params=params) as resp:
                await asyncio.sleep(1)
                return await resp.text()
#         await asyncio.sleep(0.05)
                
    async def __parse(self, text):
        pmid = None
        if 'PhraseNotFound' not in text:
            count = int(re.search(COUNT_REGEX, text).group(1))
            # Only return unique matches
            if count == 1:
                match = re.search(ID_REGEX, text)
                if match:
                    pmid = match.group(1)
        return pmid

    async def get_pmid(self, session, doi):
        params = self.params
        params['term'] = doi
        
        text = await self.__fetch(session, params)
        pmid = await self.__parse(text)
        return (doi, pmid)

    async def run(self, dois):
        tasks = []
        async with aiohttp.ClientSession(loop=self.loop) as session:
            # Create tasks with DOI
            tasks = [asyncio.ensure_future(self.get_pmid(session, doi)) for doi in dois]  

            # Await tasks and print progress
            responses = [await t for t in tqdm(asyncio.as_completed(tasks), total=len(dois))]

            return responses

In [10]:
loop = asyncio.get_event_loop()
eutils = Eutils(tool_name, email, ncbi_api_key, loop)

In [7]:
dois = articles.DOI.sample(10).unique().tolist()

In [11]:
pmids = pd.read_csv(pmid_f)
existing_dois = set(pmids.DOI.tolist())
dois = list(set(dois).difference(existing_dois))

ParserError: Error tokenizing data. C error: Expected 4 fields in line 74, saw 6


In [9]:
dois

['10.1093/jnci/djh145',
 '10.1016/j.nano.2007.11.005',
 '10.1677/joe.0.1690097',
 '10.1002/pros.20623',
 '10.1097/wco.0000000000000186',
 '10.7150/ijbs.4.223',
 '10.1021/pr500387m',
 '10.1002/pros.20227',
 '10.4049/jimmunol.1400806',
 '10.1107/s1744309110004409']

In [13]:
results = await eutils.run(dois)

In [20]:
list(set([1,2,3]).difference(set([2,3])))

[1]

In [17]:
pd.DataFrame(results, columns=["DOI", "pmid"]).to_csv(pmid_f)

,DOI,pmid
0,10.1016/j.clgc.2014.08.012,25450037
1,10.1210/endo.143.6.8846,12021174
2,10.1124/mol.107.036681,17507690
3,10.3748/wjg.v13.i11.1659,12506112
4,10.1200/jco.2015.65.3154,12506112
5,10.1371/journal.pone.0071709,12506112
6,10.1002/nbm.973,12506112
7,10.2144/000114063,12506112
8,10.4161/cbt.12.9.17682,12506112
9,10.1074/jbc.m207637200,12506112
